In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import pickle

In [23]:
#import the data
final = pd.read_csv('Resources/done.csv')
final = final.drop(columns='Unnamed: 0')

In [22]:
leveler = {'A+':3, 'A': 4, 'Rk':6, 'A-':5, 'AAA': 1, 'AA':2, 'MLB': 0}
releveler = {3:'A+', 4:'A', 6:'Rk', 5:'A-', 1: 'AAA', 2 : 'AA', 0: 'MLB'}

In [80]:
# final = final.replace({"TopLevel" : leveler})
final = final[final.PA > 30]

In [43]:
AAA = final[final.Level == 1]
AA = final[final.Level == 2]
HiA = final[final.Level == 3]
A = final[final.Level == 4]
LoA = final[final.Level == 5]
Rk = final[final.Level == 6]

In [27]:
levels = [AAA, AA, HiA, A, LoA, Rk]

In [28]:
Rk.columns

Index(['playerid', 'TopLevel', 'year', 'teamName', 'League', 'Level',
       'orgName', 'lastName', 'firstName', 'StartSeason', 'Bats', 'Throws',
       'posit', 'borndate', 'cityName', 'regionID', 'mlbid', 'PA', 'AVG',
       'SLG', 'OBP', 'OPS', 'Age', 'HT', 'WT', 'DEF', 'ISO', 'BB%', 'K%',
       'SB%', 'BABIP', 'DaysOld'],
      dtype='object')

In [76]:
#Split it into X and Y
X = final[['Level', 'DaysOld', 'DEF', 'ISO', 'BB%', 'K%', 'SB%', 'BABIP']]
y = final.TopLevel

In [81]:
X = X.fillna(0)

In [82]:
# Use train_test_split to create training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [80]:
# Create a StandardScaler model and fit it to the training data


#NOTE: I think this will be unnecessary in the final moel because of the standard deviations?

# X_scaler = MinMaxScaler().fit(X_train)

In [46]:
# Transform the training and testing data using the X_scaler
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)


#If not scaling
X_train_scaled = X_train
X_test_scaled = X_test

In [47]:
# One-hot encoding
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [48]:
# create a normal neural network with 31 inputs, 
# 31 hidden nodes in the first layer, 
# 26 in the second and 2 outputs

model = Sequential()
model.add(Dense(units=10, activation='relu', input_dim=7))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=7, activation='sigmoid'))

In [49]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [50]:
# Fit the model to the training data
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
 - 1s - loss: 1.7803 - acc: 0.3324
Epoch 2/100
 - 1s - loss: 1.7092 - acc: 0.3620
Epoch 3/100
 - 1s - loss: 1.6990 - acc: 0.3764
Epoch 4/100
 - 1s - loss: 1.6939 - acc: 0.3798
Epoch 5/100
 - 1s - loss: 1.6909 - acc: 0.3811
Epoch 6/100
 - 1s - loss: 1.6886 - acc: 0.3827
Epoch 7/100
 - 1s - loss: 1.6873 - acc: 0.3827
Epoch 8/100
 - 1s - loss: 1.6861 - acc: 0.3831
Epoch 9/100
 - 1s - loss: 1.6852 - acc: 0.3833
Epoch 10/100
 - 1s - loss: 1.6846 - acc: 0.3839
Epoch 11/100
 - 1s - loss: 1.6841 - acc: 0.3841
Epoch 12/100
 - 1s - loss: 1.6832 - acc: 0.3843
Epoch 13/100
 - 1s - loss: 1.6828 - acc: 0.3837
Epoch 14/100
 - 1s - loss: 1.6825 - acc: 0.3839
Epoch 15/100
 - 1s - loss: 1.6820 - acc: 0.3847
Epoch 16/100
 - 1s - loss: 1.6817 - acc: 0.3844
Epoch 17/100
 - 1s - loss: 1.6815 - acc: 0.3851
Epoch 18/100
 - 2s - loss: 1.6812 - acc: 0.3848
Epoch 19/100
 - 1s - loss: 1.6805 - acc: 0.3860
Epoch 20/100
 - 1s - loss: 1.6808 - acc: 0.3850
Epoch 21/100
 - 1s - loss: 1.6801 - acc: 0.3856
E

In [51]:
# See how it did.

model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

 - 0s - loss: 1.6935 - acc: 0.3856
Neural Network - Loss: 1.6935413611453511, Accuracy: 0.3855900764465332


In [83]:
# Random forest Test
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.4684118575293057

In [53]:
#Decision Tree Classifier
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.25577639751552794

In [54]:
# print(f"Predicted class: {model.predict_classes(X_test_scaled[:15])}")

# print(f"Actual Labels: {list(y_test[:15])}")

Predicted class: [6 6 6 0 3 6 0 6 6 6 6 1 6 6 6]
Actual Labels: [4, 1, 6, 3, 2, 6, 3, 6, 4, 6, 6, 3, 6, 6, 2]


In [96]:
rf.score(X_train, y_train)

0.9999530353268271

In [ ]:
rf.score(X_test, y_test)

In [74]:
Rk['Predicted'] = model.predict_classes(X)

C:\Users\alana\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [73]:
Rk[Rk['firstName'] == 'Vladimir']

,playerid,TopLevel,year,teamName,League,Level,orgName,lastName,firstName,StartSeason,...,HT,WT,DEF,ISO,BB%,K%,SB%,BABIP,DaysOld,Predicted
11262,194379,6,2010,DSL Dodgers,DSL,6,LAN,Martinez,Vladimir,4/1/2010,...,0.302082,-0.062460,0.320398,-1.476590,-0.672327,-0.390830,0.394232,0.121764,-0.583854,6
11263,194379,6,2012,DSL Dodgers,DSL,6,LAN,Martinez,Vladimir,4/1/2012,...,0.821330,-0.087418,0.309092,-0.958784,-0.002569,-1.095950,-0.978163,-0.186997,0.500845,6
11264,194379,6,2011,DSL Dodgers,DSL,6,LAN,Martinez,Vladimir,4/1/2011,...,0.507495,-0.051534,0.297036,-1.001509,0.117874,-0.886555,0.702921,-0.359484,-0.073921,6
14573,138,6,2002,Casper,Pio,6,COL,Bello,Vladimir,4/1/2002,...,-0.892749,-1.213697,-0.761067,0.077884,-0.662790,-0.272287,1.047030,0.035030,-0.283137,6
14574,138,6,2001,Casper,Pio,6,COL,Bello,Vladimir,4/1/2001,...,-0.917690,-1.417596,-0.839196,-0.030954,-0.900397,0.019504,0.929768,0.636200,-0.812516,6
17862,143617,0,2008,GCL Phillies,GCL,6,PHI,De Los Santos,Vladimir,4/1/2008,...,0.074461,-0.491667,-0.884709,1.237540,-0.701987,-0.015251,0.021678,-0.343077,0.203779,6
20515,216635,6,2018,DSL Padres,DSL,6,SDN,Echavarria,Vladimir,4/1/2018,...,0.094595,-0.159674,0.277161,0.181945,2.318989,0.676138,0.339389,0.327621,-0.129777,6
23183,217092,6,2018,DSL Dodgers,Dsl,6,LAN,Perez,Vladimir,4/1/2018,...,0.073552,-0.424860,0.389658,0.069137,1.306020,1.241083,-0.111303,0.439609,-0.518312,6
64913,216626,6,2018,DSL Colorado,DSL,6,COL,Dilone,Vladimir,4/1/2018,...,-0.077480,-0.159674,0.277161,0.414051,0.538960,-1.137948,0.228859,-0.042843,-0.362273,0
69328,203172,0,2016,Bluefield,App,6,TOR,Guerrero,Vladimir,4/1/2016,...,0.309566,0.799227,-1.069056,0.672326,0.839659,-1.123013,0.747646,-0.423869,-2.318970,0


In [119]:
#Saved the model
model.save("Rk_Model.h5")

In [139]:
final.to_csv("RkPredicts.csv")

In [94]:
filename = "Models/randomforest.h5"

In [95]:
pickle.dump(rf, open(filename, 'wb'))